In [ ]:
import codecs
import json, os, sys
import pickle
import csv
import time
import datetime
import collections
import numpy as np
import compmusic
from compmusic import dunya 
from compmusic import dunya as dn
from compmusic.dunya import hindustani as hi
from compmusic.dunya import docserver as ds
from compmusic import musicbrainz


In [ ]:
dunya.set_token('2bd80bef6e90945815fe53660eb43faebe2346ac')
features_dunya_all = [{'type': 'pitch', 'subtype': 'pitch', 'extension': '.pitch', 'version': 'noguessunv'},
                         {'type': 'ctonic', 'subtype': 'tonic', 'extension': '.tonic', 'version': '0.3'},
                         {'type': 'sama-manual', 'subtype': None, 'extension': '.sama', 'version': None},
                         {'type': 'sections-manual', 'subtype': None, 'extension': '.sections', 'version': None},
                         {'type': 'tempo-manual', 'subtype': None, 'extension': '.tempo', 'version': None},
                         {'type': 'pitch-vocal', 'subtype': None, 'extension': '.mpitch', 'version': None},
                         {'type': 'mphrases-manual', 'subtype': None, 'extension': '.mphrases', 'version': None},
                         {'type': 'sections-manual-p', 'subtype': None, 'extension': '.sections_p', 'version': None},
                         {'type': 'bpm-manual', 'subtype': None, 'extension': '.bpm', 'version': None}
                         ]

In [ ]:
def txt(features,mbid,dire):
    for feature in features:
        missingData = collections.Counter()
        missingData[feature['type']] = 0
        try:
            content = ds.file_for_document(mbid, feature['type'], feature['subtype'], version=feature['version'])
            out_file = os.path.join(dire, mp3_filename.replace('.mp3','.{}.txt'.format(feature['type'])))
            if feature['type'] == 'pitch':
                content = json.loads(content.decode())
                content = np.array(content)
                np.savetxt(out_file, content, fmt='%.7f', delimiter='\t')
            elif feature['type'] == 'sections-manual' or feature['type'] == 'sections-manual-p':
                save_sections(content.decode(), out_file)
            else:
                with open(out_file, 'w') as fp:
                    fp.write(content.decode())
        except dn.HTTPError:
            print('Does not have ',feature['type'],' content for :',mbid)
            missingData[feature['type']] += 1
            print('no')
    return dict(missingData)

In [ ]:
instrument_ids_list=[10] #Flute, mandolin, saxophone
h=0

for iid in instrument_ids_list:
    for artist in dunya.carnatic.get_instrument(iid)['artists']:
        dataDir=dunya.carnatic.get_instrument(iid)['name']
        os.makedirs(dataDir,exist_ok=True)
        os.makedirs(os.path.join(dataDir,'_json'),exist_ok=True)
        os.makedirs(os.path.join(dataDir,'_txt'),exist_ok=True)
        concerts = dunya.carnatic.get_artist(artist['mbid'])['concerts']
        for concert in concerets:      
            for i in compmusic.dunya.carnatic.get_concert(concert['mbid'])['recordings']:  
                try:
                    print(len(i))
                    mp3_filename=dunya.carnatic.download_mp3(i['mbid'],  dataDir)
                    json_file=mp3_filename.replace('.mp3','.json')
                    with open(os.path.join(os.path.join(dataDir,'_json'),json_file), 'w') as outfile:
                        json.dump(dunya.carnatic.get_recording(i['mbid']), outfile)
                    txt(features_dunya_all,i['mbid'],os.path.join(dataDir,'_txt'))
                except dn.HTTPError:
                    pass

In [ ]:
#hindustani
instrument_ids_list=['0fe1a768-45ba-49e4-8363-14db8e73ca85','f689271c-37bc-4c49-92a3-a14b15ee5d0e']
h=0
for iid in instrument_ids_list:
    for artist in dunya.hindustani.get_instrument(iid)['artists']:
        dataDir=dunya.hindustani.get_instrument(iid)['name']
        os.makedirs(dataDir,exist_ok=True)
        os.makedirs(os.path.join(dataDir,'_json'),exist_ok=True)
        os.makedirs(os.path.join(dataDir,'_txt'),exist_ok=True)
        releases = dunya.hindustani.get_artist(artist['mbid'])['releases']    
        for release in releases:
            for i in compmusic.dunya.hindustani.get_release(release['mbid'])['recordings']:  
                try:
                    mp3_filename=dunya.hindustani.download_mp3(i['mbid'],  dataDir)
                    json_file=mp3_filename.replace('.mp3','.json')
                    with open(os.path.join(os.path.join(dataDir,'_json'),json_file), 'w') as outfile:
                        json.dump(dunya.hindustani.get_recording(i['mbid']), outfile)
                    txt(features_dunya_all,i['mbid'],os.path.join(dataDir,'_txt'))
                except dn.HTTPError:
                    pass 

In [ ]:
a=dunya.hindustani.get_instrument('9290b2c1-97c3-4355-a26f-c6dba89cf8ff')['artists']
for i in a:
    print(dunya.hindustani.get_artist(i['mbid'])['releases'],'\n')